## Generate an initial structure

In [ ]:
smiles = "C=CCC1CO1"

In [22]:
from rdkit import Chem
from rdkit.Chem import Draw

import sys

sys.path.append("../src/protopilot")

from protopilot.io import optimize

print(
    "Copy the bash command and run it in the terminal to perform the optimization.\nMake sure to check that the structure is correct!"
)
par_dir = optimize.write(
    smiles_string=smiles,
)

mol = Chem.MolFromSmiles(smiles)
img = Draw.MolToImage(
    mol,
    molsPerRow=1,
    subImgSize=(200, 200),
)

display(img)

Copy the bash command and run it in the terminal to perform the optimization.
Make sure to check that the structure is correct!


[16:54:21] SMILES Parse Error: Failed parsing SMILES 'CdblCCC1CO1' for input: 'CdblCCC1CO1'


ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.AddHs(NoneType)
did not match C++ signature:
    AddHs(RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, boost::python::api::object onlyOnAtoms=None, bool addResidueInfo=False)
    AddHs(RDKit::ROMol mol, RDKit::MolOps::AddHsParameters params, boost::python::api::object onlyOnAtoms=None)

## Once the initial structure has finished optimization (up to REVDSD)

1. Indicate a smiles string to reference the transition structure. Then, review the images and indicate involved atom indices for the reaction.

In [1]:
trans_smiles = "[CH2]C1~CO1"

In [11]:
from pathlib import Path
import py3Dmol

trans_smiles = trans_smiles.replace("/", "~")
revdsd_matches = [path for path in (par_dir / "REVDSD").rglob("REVDSD.xyz")]
if len(revdsd_matches) > 1:
    raise ValueError(
        f"Multiple XTB.allxyz files found in {par_dir}/REVDSD. Please check the directory."
    )
else:
    inter_xyz = Path(revdsd_matches[0])

natoms = int(inter_xyz.read_text().split()[0])
natoms = int(inter_xyz.read_text().split()[0])

# Draw transition state structure
mol = Chem.MolFromSmiles(trans_smiles)
img = Draw.MolToImage(
    mol,
    molsPerRow=1,
    subImgSize=(600, 600),
)
display(img)

# Animate optimized structure
viewer = py3Dmol.view()
viewer.addModel(inter_xyz.read_text(), "xyz")
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.3}})

for i in range(natoms):
    viewer.addLabel(
        i,
        {
            "backgroundOpacity": 0,
            "fontColor": "blue",
            "alignment": "center",
            "inFront": True,
        },
        {"index": i},
    )

viewer.zoomTo()
viewer.show()

NameError: name 'par_dir' is not defined

In [1]:
reaction_indices = [1, 2]  # [indice_1, indice_2]
charge = 0
multiplicity = 2  # 1 if neutral, 2+ if radical
min_scan_dist = 0.1  # minimum distance for scan
max_scan_dist = 1.5  # maximum distance for scan

In [6]:
import sys
from pathlib import Path
import numpy as np

sys.path.append(str(Path.home() / "C5O-Kinetics"))
from src.protopilot.io.transition import write_scan

mol = Chem.MolFromXYZFile(str(inter_xyz))

n_steps = abs(max_scan_dist - min_scan_dist) // 0.05 + 1

# --- Get 3D coordinates
with open(inter_xyz) as f:
    lines = f.readlines()
coords = []
for line in lines[2:]:  # skip first two lines of xyz
    parts = line.split()
    if len(parts) >= 4:
        coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
coords = np.array(coords)
bond_len = np.linalg.norm(
    coords[reaction_indices[0], :] - coords[reaction_indices[1], :]
)

rxn_coordinates = f"B {reaction_indices[0]} {reaction_indices[1]} = {(bond_len + min_scan_dist):.2f}, {(bond_len + max_scan_dist):.2f}, {n_steps}\n"
print(rxn_coordinates)
write_scan(
    intermediate_xyz=str(inter_xyz),
    trans_smiles=trans_smiles,
    scan_coordinates=f"B_{reaction_indices[0]} {reaction_indices[1]}",
    rxn_coordinates=rxn_coordinates,
    charge=charge,
    multiplicity=multiplicity,
)

NameError: name 'inter_xyz' is not defined